<a href="https://colab.research.google.com/github/brandonmoss124/mgmt467-analytics-portfolio/blob/main/Lab8_Streaming_Pipeline_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup: install Pub/Sub client library in Colab
!pip install -q google-cloud-pubsub functions-framework

from google.colab import auth  # comment out if not using Colab
auth.authenticate_user()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.4 MB/s eta 0:00:00


# Lab 8: Building a Streaming Pipeline (Pub/Sub + Dataflow)

This notebook documents your steps to convert the batch pipeline from Lab 7
into a streaming pipeline using Pub/Sub and a Dataflow template.


## 1. Pub/Sub Topic Setup



## 2. Modify Cloud Function to Publish to Pub/Sub



In [2]:
from google.cloud import pubsub_v1
import json
import os
import logging
import functions_framework

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# TODO: update this to your actual project ID and topic name
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT", "proven-agility-477721-q9")
TOPIC_ID = "live-data-stream"

publisher = pubsub_v1.PublisherClient()
TOPIC_PATH = publisher.topic_path(PROJECT_ID, TOPIC_ID)

def publish_weather(weather_dict):
    """Publish a weather dict as JSON to Pub/Sub."""
    data_str = json.dumps(weather_dict)
    future = publisher.publish(TOPIC_PATH, data=data_str.encode("utf-8"))
    message_id = future.result()
    logger.info(f"Published message ID: {message_id}")

@functions_framework.http
def ingest_weather_producer(request):
    """HTTP Cloud Function that fetches weather and publishes to Pub/Sub.

    NOTE: This expects you to have defined `fetch_weather_lafayette()`
    (from Lab 7) in the same source file.
    """
    weather_json = fetch_weather_lafayette()
    publish_weather(weather_json)
    return ("OK: message published", 200)

## 3. Validate Streaming Data in BigQuery

```sql
SELECT *
FROM `your-project.superstore_data.realtime_weather_streaming`
ORDER BY dt_utc DESC
LIMIT 10;
```


4. Challenge: Apache Beam Concepts Prompt

My Prompt to Gemini:

"Explain Apache Beam's three core concepts Pipeline, PCollection, and ParDo using an assembly line or factory analogy."

Gemini’s Response:

Think of Apache Beam like a factory assembly line:

- **Pipeline** is the entire assembly line in the factory. It defines the overall path that items (data) will follow from start to finish.
- **PCollection** is the set of items moving along the conveyor belt — the data flowing through the pipeline.
- **ParDo** is like a worker or robot at a specific station on the line that performs an operation on each item (for example, cleaning, painting, or inspecting each product).

Together, the Pipeline (line), PCollection (items), and ParDo (workers) describe how data is created, transformed, and passed along in a streaming or batch system.
